In [ ]:
pip install datasets

In [37]:
import pandas as pd
import numpy as np
import torch
import torchtext
import torchvision
import tarfile
import datasets as ds
from torch.hub import load_state_dict_from_url
from torch.nn import Embedding
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [ ]:
qqp_dataset = ds.load_dataset('glue', 'qqp', split = 'train')  
print(qqp_dataset.shape)
qqp_df = qqp_dataset.to_pandas()
df_qqp = qqp_df.head(100)
df_qqp

In [ ]:
questions_list = list(df_qqp['question1'])
splitted = [i.split() for i in questions_list]
splitted

In [ ]:
cola_dataset = ds.load_dataset('glue', 'cola', split = 'train') 
print(cola_dataset.shape)
cola_df = cola_dataset.to_pandas()
df_cola = cola_df.head(100)
print(df_cola.shape)

In [ ]:
cola_sentences = df_cola['sentence']
cola_list = list(cola_sentences)
cola_split = [i.split() for i in cola_list]
cola_club = []
for i in cola_split:
  for j in i:
    cola_club.append(j)
cola_club

In [43]:
freq = 2
emb_dimensions = 100
Language_model_build_Ngram = [
    (
        [cola_club[i - j - 1] for j in range(freq)],
        cola_club[i]
    )
    for i in range(freq, len(cola_club))
]
# Print the first 3, just so you can see what they look like.
print(Language_model_build_Ngram[:3])

terminology = set(cola_club)
dict_index = {word: i for i, word in enumerate(terminology)}


class LM_Ngram(nn.Module):

    def __init__(self, terminology_size, emb_dimensions, freq):
        super(LM_Ngram, self).__init__()
        self.embd = nn.Embedding(terminology_size, emb_dimensions)
        self.lin1 = nn.Linear(freq * emb_dimensions, 128)
        self.lin2 = nn.Linear(128, terminology_size)

    def forward(self, inputs):
        embds = self.embd(inputs).view((1, -1))
        out = F.relu(self.lin1(embds))
        out = self.lin2(out)
        model_log = F.log_softmax(out, dim=1)
        return model_log


wastage = []
loss_func = nn.NLLLoss()
model_build = LM_Ngram(len(terminology), emb_dimensions, freq)
optimum_build = optim.SGD(model_build.parameters(), lr=0.001)

for e in range(10):
    agg_loss = 0
    for context, target in Language_model_build_Ngram:

        # Step 1. Prepare the inputs to be passed to the model_build (i.e, turn the words
        # into integer indices and wrap them in tensors)
        term_index_log = torch.tensor([dict_index[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model_build.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        model_log = model_build(term_index_log)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_func(model_log, torch.tensor([dict_index[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimum_build.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        agg_loss += loss.item()
    wastage.append(agg_loss)
print(wastage)  # The loss decreased every iteration over the training info!

# To get the embedding of a particular word, e.g. "beauty"
print(model_build.embd.weight[dict_index["friends"]])


[(['friends', 'Our'], "won't"), (["won't", 'friends'], 'buy'), (['buy', "won't"], 'this')]
[3588.069879055023, 3480.4721961021423, 3368.556111097336, 3249.9172459840775, 3135.2240887880325, 3040.3954313099384, 2962.2679595798254, 2891.4134205579758, 2823.377431035042, 2756.0928336530924]
tensor([ 0.2396,  0.0502,  0.9585,  0.2359, -1.3063,  0.4543,  0.5654, -1.0022,
        -1.5026, -1.5885,  0.0306,  0.2366, -0.8308, -1.3930, -0.0421, -0.3654,
        -0.6799,  0.7382, -1.1749,  0.2314, -0.8781,  0.0135, -2.2154,  0.1822,
        -1.6944,  1.2787,  0.6896, -0.3323,  0.4410, -0.0276, -0.3023, -1.1430,
         1.1011, -0.7633, -0.6471,  0.6249, -1.0654, -0.4269, -0.4138, -0.7262,
         0.3140,  0.6562, -0.2120,  1.0820,  0.0158, -0.9244,  0.1684,  0.3090,
         0.6645,  0.1435, -1.0088, -0.2322,  1.1397, -0.9765,  0.4586, -1.0782,
         1.0066, -0.5182,  0.7757,  0.0117,  0.0998, -0.1655,  0.7795, -0.5502,
        -2.4609, -0.9725,  0.4809,  0.9322, -0.3171, -1.2726, -0.0961, 

In [44]:
terminology = set(cola_club)
terminology_size = len(terminology)

dict_index = {word: i for i, word in enumerate(terminology)}
info = []
for i in range(freq, len(cola_club) - freq):
    context = (
        [cola_club[i - j - 1] for j in range(freq)]
        + [cola_club[i + j + 1] for j in range(freq)]
    )
    target = cola_club[i]
    info.append((context, target))
print(info[:5])


class module_class(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# Create your model_build and train. Here are some functions to help you make
# the info ready for use by your module.


def term_vector(context, dict_index):
    indexes = [dict_index[w] for w in context]
    return torch.tensor(indexes, dtype=torch.long)


term_vector(info[0][0], dict_index)

[(['friends', 'Our', 'buy', 'this'], "won't"), (["won't", 'friends', 'this', 'analysis,'], 'buy'), (['buy', "won't", 'analysis,', 'let'], 'this'), (['this', 'buy', 'let', 'alone'], 'analysis,'), (['analysis,', 'this', 'alone', 'the'], 'let')]


tensor([122, 179, 243,  61])

Reference - https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html 